In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/advance-house-price-predicitons/sample_submission.csv
/kaggle/input/advance-house-price-predicitons/data description.txt
/kaggle/input/advance-house-price-predicitons/formulatedtest.csv
/kaggle/input/advance-house-price-predicitons/train.csv
/kaggle/input/advance-house-price-predicitons/test.csv


In [2]:
import cuml 
import cupy as cp
import cudf as cd
import seaborn as sns

In [3]:
df_train = cd.read_csv("../input/advance-house-price-predicitons/train.csv")
df_train= df_train.select_dtypes(include='number')
df_train=df_train.fillna(method ='pad')

df_test = cd.read_csv("../input/advance-house-price-predicitons/test.csv")
df_test= df_test.select_dtypes(include='number')
df_test=df_test.fillna(method ='pad')


In [4]:
df_test.head(10)

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,1461,20,80.0,11622,5,6,1961,1961,0.0,468.0,...,730.0,140,0,0,0,120,0,0,6,2010
1,1462,20,81.0,14267,6,6,1958,1958,108.0,923.0,...,312.0,393,36,0,0,0,0,12500,6,2010
2,1463,60,74.0,13830,5,5,1997,1998,0.0,791.0,...,482.0,212,34,0,0,0,0,0,3,2010
3,1464,60,78.0,9978,6,6,1998,1998,20.0,602.0,...,470.0,360,36,0,0,0,0,0,6,2010
4,1465,120,43.0,5005,8,5,1992,1992,0.0,263.0,...,506.0,0,82,0,0,144,0,0,1,2010
5,1466,60,75.0,10000,6,5,1993,1994,0.0,0.0,...,440.0,157,84,0,0,0,0,0,4,2010
6,1467,20,75.0,7980,6,7,1992,2007,0.0,935.0,...,420.0,483,21,0,0,0,0,500,3,2010
7,1468,60,63.0,8402,6,5,1998,1998,0.0,0.0,...,393.0,0,75,0,0,0,0,0,5,2010
8,1469,20,85.0,10176,7,5,1990,1990,0.0,637.0,...,506.0,192,0,0,0,0,0,0,2,2010
9,1470,20,70.0,8400,4,5,1970,1970,0.0,804.0,...,525.0,240,0,0,0,0,0,0,4,2010


In [5]:
df_train.shape

(1460, 38)

In [6]:
from cuml.preprocessing import StandardScaler
X=df_train.loc[:, df_train.columns.drop('SalePrice')]
Y = df_train['SalePrice']
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
from cuml.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y,test_size=0.25,random_state=29)


In [7]:
print(X.shape,X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(1460, 37) (1095, 37) (365, 37) (1095,) (365,)


In [8]:
algorithm = ['svd', 'eig', 'qr', 'svd-qr', 'svd-jacobi']


In [9]:
from cuml import LinearRegression
err = 9999999999
final_lr = None
for alg in algorithm:
    print("Algorithm:")
    print(alg)
    lr = LinearRegression(fit_intercept = True, normalize = False, algorithm = alg)
    reg = lr.fit(X_train,Y_train)
    preds = lr.predict(X_test)
    Y_test=Y_test.astype(cp.float64)
    print("MSE:",cuml.metrics.regression.mean_squared_error(Y_test,preds))
    if err>=cuml.metrics.regression.mean_squared_error(Y_test,preds):
        err = cuml.metrics.regression.mean_squared_error(Y_test,preds)
        final_lr = lr
    print("R2 Score:",cuml.metrics.regression.r2_score(Y_test,preds))
    print("MAE:",cuml.metrics.regression.mean_absolute_error(Y_test,preds))
    print()

Algorithm:
svd
MSE: 744626651.5897641
R2 Score: 0.8448279946908837
MAE: 19778.97342354071

Algorithm:
eig
MSE: 997574699.8426546
R2 Score: 0.7921164031803876
MAE: 24116.10381288953

Algorithm:
qr
MSE: 764905697.6163703
R2 Score: 0.8406020645136786
MAE: 20117.373678685777

Algorithm:
svd-qr
MSE: 744626651.5893263
R2 Score: 0.8448279946909749
MAE: 19778.973423535397

Algorithm:
svd-jacobi
MSE: 744626651.5897641
R2 Score: 0.8448279946908837
MAE: 19778.97342354071



In [10]:
Y_new_test=scaler.transform(df_test)
final_prediced=final_lr.predict(Y_new_test)

In [11]:
test_id = df_test[['Id']]
output = cd.DataFrame()
output["Id"] = test_id
output["SalePrice"] = cd.DataFrame(final_prediced)
output.to_csv('submission.csv', index=False)